In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iris-dataset/iris.csv


TODO: Go through this in more detail to see how the two implementations vary, and why they give different results.

Note: Figured out where the input is stored by default...  Note the directory structure.

Since I used a dataset in Kaggle with the same info but slightly different format I had to edit some details to get this all to work.

In [1]:
!ls /kaggle/input/iris-dataset

iris.csv


In [2]:
!head /kaggle/input/iris-dataset/iris.csv

sepal_length,sepal_width,petal_length,petal_width,species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa


Note: Tried to add %debug, then force a break by adding a divide by zero.  The code did halt, but I was unable to get the variables from the fit function. I will probably need to dig into the Python command line debugger to figure out how to do such things, as I cannot find a visual debugger for Kaggle.

In [8]:
import numpy as np
import random

# Perceptron with numpy + pandas
# The guy is great, will definetly give a shoutout
class Perceptron():
    '''
        Perceptron Learning Algorithm that can be train using a 
        fit and predict methodology with numpy
    '''
    # I added the theta as otherwise the activation function was picking 1.0 as an output with any non-zero
    # input.  This does not seem to improve results in this simple example.
    def __init__(self, theta):
        self.weights = []
        self.theta = theta
        
    def fit(self, X, y, learning_rate = 0.01, num_iteration = 100, feedback = 100):
        
        # Using the shape of the input features, determine the number of rows and columns
        (num_row, num_feature) = X.shape
        
        # Randomly initalize the weights with a uniform distribution [0,1), to one more than 
        # the number of features, to account for the bias
        self.weights = np.random.rand(num_feature+1) 

        # Launch the training algorithm
        for i in range(num_iteration):
            
            # Stochastic Gradient Descent
            # Stochastic as it picks a random row rather than all rows
            r_i = random.randint(0,num_row-1)
            row = X[r_i,:] # take the random sample from the dataset
            yhat = self.predict(row)
            error = (y[r_i] - yhat) # estimate of the gradient (estimate because algorithm is stochastic)
            self.weights[0] = self.weights[0] + learning_rate*error*1 # first weight is the bias
            
            # Force an error
            result = 5 / 0

            # Update all parameters after bias
            # I changed this to start with range 1 to properly index weights. This works, (and
            # more closely represents the original code that this was taken from) 
            for f_i in range(1,num_feature+1):
                self.weights[f_i] = self.weights[f_i] + learning_rate*error*row[f_i-1]
                
            if i % feedback == 0:
                total_error = 0
                for r_i in range(num_row):
                    row = X[r_i,:]
                    yhat = self.predict(row)
                    error = (y[r_i] - yhat)
                    total_error = total_error + error**2
                mean_error = total_error/num_row
                print(f"Iteration {i} with error = {mean_error}")
        
    def predict(self, row):
            
        # The activation start with the bias at weights == 0
        activation = self.weights[0]
        
        # We iterate over the weights and the features in the given row
        for weight, feature in zip(self.weights[1:], row):
            activation = activation + weight*feature
            
        # Heaviside Step Function Activation
        if activation >= self.theta:
            return 1.0
        return 0.0

In [3]:
import pandas as pd

# Data sets
df = pd.read_csv('/kaggle/input/iris-dataset/iris.csv')

df.describe(include='all')

,sepal_length,sepal_width,petal_length,petal_width,species
count,150.000000,150.000000,150.000000,150.000000,150
unique,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,NaN,setosa
freq,NaN,NaN,NaN,NaN,50
mean,5.843333,3.054000,3.758667,1.198667,NaN
std,0.828066,0.433594,1.764420,0.763161,NaN
min,4.300000,2.000000,1.000000,0.100000,NaN
25%,5.100000,2.800000,1.600000,0.300000,NaN
50%,5.800000,3.000000,4.350000,1.300000,NaN
75%,6.400000,3.300000,5.100000,1.800000,NaN


In [4]:
# Do a one hot encoding
df = pd.get_dummies(df,prefix='species')
X = df[['sepal_length','sepal_width','petal_length','petal_width']]
# Convert the matrix X (note the capitalization) to numpy compatible matrix (no named rows/columns)
X = X.to_numpy()

df.describe(include='all')

,sepal_length,sepal_width,petal_length,petal_width,species_setosa,species_versicolor,species_virginica
count,150.000000,150.000000,150.000000,150.000000,150,150,150
unique,NaN,NaN,NaN,NaN,2,2,2
top,NaN,NaN,NaN,NaN,False,False,False
freq,NaN,NaN,NaN,NaN,100,100,100
mean,5.843333,3.054000,3.758667,1.198667,NaN,NaN,NaN
std,0.828066,0.433594,1.764420,0.763161,NaN,NaN,NaN
min,4.300000,2.000000,1.000000,0.100000,NaN,NaN,NaN
25%,5.100000,2.800000,1.600000,0.300000,NaN,NaN,NaN
50%,5.800000,3.000000,4.350000,1.300000,NaN,NaN,NaN
75%,6.400000,3.300000,5.100000,1.800000,NaN,NaN,NaN


The following code produces this type of data, if it runs (but the list might be longer than I want to show here):

Numpy Array
----------
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]

In [31]:
#print('\nNumpy Array\n----------\n', X)

In [5]:
y = df.loc[:, 'species_versicolor']
y = y.to_numpy()

# Shuffle the two dataset in unison by creating a random index vector and applying it to both
perm = np.random.permutation(len(X))
X = X[perm]
y = y[perm]
#print(perm)

In [9]:
# Given that fit is called right after the Perceptron is created, some parameters could be passed to the initializer.
clf = Perceptron(0.2)
clf.fit(X, y, num_iteration = 40, feedback = 2)

ZeroDivisionError: division by zero

Python Debugger
I've spent a bunch of time searching and still have not found a way to get an interactive debugger with breakpoints. It is possible to get a debugger after an error has killed the app. Of course this is no where near as useful or powerful. Fascinating that pretty much no one points this out in all of the happy videos and articles talking about debugging notebooks. These tutorials spend the vast majority of their time disucssing their individual issues, rather than debugging in general. Arrrrrgggh!

For non-online notebooks you can use VSCode...

I forced an error with a simple line: result = 5 / 0

Commands
u - back one step
d - forward one step
p - print something (variable or expression)
q - quit


In [ ]:
%debug

> /tmp/ipykernel_33/1206212605.py(38)fit()
     36 
     37             # Force an error
---> 38             result = 5 / 0
     39 
     40             # Update all parameters after bias



ipdb>  p error


-1.0


ipdb>  u


> /tmp/ipykernel_33/3169435832.py(3)<module>()
      1 # Given that fit is called right after the Perceptron is created, some parameters could be passed to the initializer.
      2 clf = Perceptron(0.2)
----> 3 clf.fit(X, y, num_iteration = 40, feedback = 2)



ipdb>  d


> /tmp/ipykernel_33/1206212605.py(38)fit()
     36 
     37             # Force an error
---> 38             result = 5 / 0
     39 
     40             # Update all parameters after bias



Results are not great, and jump back and forth...  I suspect something is wrong with the algorithm, or with the error calculation.

In [37]:
import random

class Perceptron():
    '''
        Perceptron Learning Algorithm that can be train using a 
        fit and predict methodology, without any library
    '''
    
    def __init__(self):
        self.weights = []
        
    def fit(self, X, y, learning_rate = 0.01, num_iteration = 100):
        
        num_row = len(X)
        num_feature = len(X[0]) # Here we assume that we have a rectangular matrix
        
        # Randomly initalize the weights
        for i in range(num_feature+1):
            self.weights.append(random.uniform(0,1))
        
        # Launch the training algorithm
        
        for i in range(num_iteration):
            
            # Stochastic Gradient Descent
            r_i = random.randint(0,num_row-1)
            row = X[r_i]
            yhat = self.predict(row)
            error = (y[r_i] - yhat)
            self.weights[0] = self.weights[0] + learning_rate*error

#            for f_i in range(num_feature):
#                self.weights[f_i] = self.weights[f_i] + learning_rate*error*row[f_i]
            # I changed this to start with range 1 to properly index weights. This works, (and
            # more closely represents the original code that this was taken from) 
            for f_i in range(1,num_feature+1):
                self.weights[f_i] = self.weights[f_i] + learning_rate*error*row[f_i-1]
                
            if i % 100 == 0:
                total_error = 0
                for r_i in range(num_row):
                    row = X[r_i]
                    yhat = self.predict(row)
                    error = (y[r_i] - yhat)
                    total_error = total_error + error**2
                mean_error = total_error/num_row
                print(f"Iteration {i} with error = {mean_error}")
        
    def predict(self, row):
            
        # The activation start with the bias at weights == 0
        activation = self.weights[0]
        
        # We iterate over the weights and the features in the given row
        for weight, feature in zip(self.weights[1:], row):
            activation = activation + weight*feature
            
        if activation >= 0.0:
            return 1.0
        return 0.0

In [39]:
import csv

def permute_together(X,y):
    '''
        Helper function to permute (shuffle) a matrix and a vector together
    '''
    
    perm_X = []
    perm_y = []
    while len(X) != 0 and len(y) != 0:
        
        perm_id = random.randint(0,len(X)-1)
        perm_X.append(X.pop(perm_id))
        perm_y.append(y.pop(perm_id))
        
    return (perm_X, perm_y)

class DataFrame():
    '''
        Simple dataframe to mimick the pandas library
    '''
    def __init__(self):
        self.header = [];
        self.X = []
        self.y = []
        
    def clean_string(self,string):
        '''
            Dummy function to clean up the iris dataset from (")
        '''
        return string.replace('"', '')
    

    def get_encoded_labels(self, target):
        '''
            Encode with 1 or 0 the y vector if it match our target variable
        '''
        labels = []
        for label in self.y:
            if label == target:
                labels.append(1)
            else:
                labels.append(0)
        return labels
    
    def read_csv(self, filename):
        '''
            Read the iris dataset CSV file and populate the header, the X and the y variables
            needed for the perceptron
        '''
        with open(filename, newline='', encoding="utf-8-sig") as csvfile:
            csvreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
            
            index = -1
            for row in csvreader:
                
                # We have the header
                if index == -1:
                    self.header = [self.clean_string(s) for s in row[0].split(',')]
                    index += 1
                    continue
                
                # Here is the data
                x = []
                target = None
                data = row[0].split(',')
                for i in range(len(data)-1):
                    x.append(float(data[i]))
                
                # Last item in the csv will be the target
                self.y.append(self.clean_string(data[len(data)-1]))
                self.X.append(x)
                
                index += 1
        
                

# Data sets
df = DataFrame()
df.read_csv('/kaggle/input/iris-dataset/iris.csv')

X = df.X
y = df.get_encoded_labels('Versicolor') # encoding for 0 and 1

# Shuffle the two dataset in unison
X,y = permute_together(X,y)

clf = Perceptron()
clf.fit(X,y, num_iteration = 1000)

Iteration 0 with error = 1.0
Iteration 100 with error = 0.0
Iteration 200 with error = 0.0
Iteration 300 with error = 0.0
Iteration 400 with error = 0.0
Iteration 500 with error = 0.0
Iteration 600 with error = 0.0
Iteration 700 with error = 0.0
Iteration 800 with error = 0.0
Iteration 900 with error = 0.0
